In [ ]:
import wave
import numpy as np
import pylab as plt
from scipy.signal import firwin
%matplotlib inline
np.set_printoptions(precision=4)

In [ ]:
cMatrix = [
 0.000045, -0.000936,  0.004049, -0.011926,  0.029106, -0.066340,  0.180800,  0.952919, -0.122473,  0.050329, -0.021920,  0.008548, -0.002638,  0.000476, 
 0.000164, -0.008294,  0.034886, -0.100099,  0.237076, -0.499944,  1.288540, -0.558971, -0.574282,  0.260273, -0.110664,  0.039675, -0.010165,  0.000669, 
 0.004461, -0.000724, -0.024895,  0.100726, -0.258546,  0.376681,  0.446216, -1.663592,  1.469000, -0.649437,  0.283180, -0.107185,  0.030675, -0.003629, 
-0.005606,  0.014003, -0.025966,  0.040406, -0.073976,  0.370403, -0.962637,  1.147171, -0.721917,  0.316915, -0.142048,  0.056324, -0.017396,  0.002484,  
]
NTAPS = 14
ORDER = 3

In [ ]:
def sample_to_int(data):
        d = int.from_bytes(data, byteorder='little', signed=True)
        return d

def int_to_sample(data):
        d = int(data)
        if(d>32765):
            d = 32765
        if(d<-32765):
            d = -32765
        return d.to_bytes(2, byteorder='little', signed=True)

In [ ]:
out_sample = 0
sample_inc = 1.001

In [ ]:
fname = "sine.wav"
f = wave.open(fname, 'rb')

f_out = wave.open("wave_out.wav", "wb")
f_out.setframerate(48000)
f_out.setsampwidth(2)
f_out.setnchannels(1)

In [ ]:
audio_delay_line = np.zeros(NTAPS)
delay_line_sample = -NTAPS

In [ ]:
def c(ixTap, ixPower):
    return cMatrix[ixPower * (NTAPS) + ixTap]

In [ ]:
def get_imp(tap, frac):
    horner_sum = c(tap, ORDER)
    for i in range(ORDER-1, -1, -1,):
        horner_sum *= frac
        horner_sum += c(tap, i)
    return horner_sum

In [ ]:
def get_c(frac):
    return [get_imp(i, frac) for i in range(NTAPS)]

In [ ]:
def process_sample(frac):
    fir_c = get_c(frac)
    v = np.sum(audio_delay_line*fir_c)
    return v

In [ ]:
while(True):
    data = f.readframes(1)
    audio_delay_line = np.roll(audio_delay_line, 1)
    audio_delay_line[0] = sample_to_int(data)
    delay_line_sample += 1
    if(len(data)==0):
        break
    while(out_sample < delay_line_sample):
        frac = out_sample % 1
        v = process_sample(frac)
        f_out.writeframes(int_to_sample(v))
        out_sample += sample_inc

In [ ]:
f.close()
f_out.close()